In [8]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")


max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

successful_launches_df = spacex_df[spacex_df['class'] == 1]
sites = successful_launches_df['Launch Site'].tolist()

# insert the list to the set
list_set = set(sites)
# convert the set to the list
sites = (list(list_set))

print(sites)



successful_launches_df = successful_launches_df.groupby(['Launch Site'])['Launch Site'].count()

successful_launches_df = pd.DataFrame({'Launch Site':successful_launches_df.index, 'Successes':successful_launches_df.values})



# Create a dash application
app = dash.Dash(__name__)


['KSC LC-39A', 'CCAFS LC-40', 'CCAFS SLC-40', 'VAFB SLC-4E']


In [9]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                  dcc.Dropdown(
                                    options=[{'label':'All sites', 'value':'ALL'} ]+[{'label':x, 'value':x} for x in sites],
                                    value='ALL',
                                    placeholder="All",
                                    searchable=True,
                                    id='site-dropdown'
                                    ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(
                                    id='success-pie-chart', 
                                    figure=px.pie(
                                        successful_launches_df,
                                        values='Successes',
                                        names='Launch Site',
                                        title="Number of Successes by Site"))
                                ),
                                html.Br(),
                                
                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                min=min_payload, max=max_payload, step=1000,
                                marks={0: '0',
                                        100: '100'},
                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(
                                        id='success-payload-scatter-chart',
                                        figure = px.scatter(
                                                spacex_df,
                                                x="Payload Mass (kg)",
                                                y="class"))
                                ),
                        ])

In [ ]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output('success-pie-chart', 'children'),
    Input('site-dropdown', 'value'))
def getpiechart(selected_site):
    if selected_site=='ALL':
        pi_figure=px.pie(
                        successful_launches_df,
                        values='Successes',
                        names='Launch Site',
                        title="Number of Successes by Site")
    else:
       successful_launches_site_df = successful_launches_df[successful_launches_df['Launch Site'] == selected_site]
       pi_figure=px.pie(
                        suuccessful_launches_site_df,
                        values='Successes',
                        names='Launch Site',
                        title="Number of Successes by Site")
    return pi_figure


# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output


@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('payload-slider', 'value'), Input('site-dropdown', 'value')])
def getscatterchart(site_dropdown, payload):
    scatter_df = spacex_df.loc((spacex_df['Launch Site'==site_dropdown])&(spacex_df['Payload Mass (kg)'>payload[0]])&(spacex_df['Payload Mass (kg)'<payload[1]]))
    #df_mod = df_mod.loc[(df_mod['Payload Mass (kg)'] >= payload[0]) & (df_mod['Payload Mass (kg)'] <= payload[1])
    scatter_chart = dcc.Graph(
                        figure = px.scatter(
                                scatter_df,
                                x="Payload Mass (kg)",
                                y="class"))
    return scatter_chart

# Run the app
if __name__ == '__main__':
    app.run_server()

[2024-02-28 19:33:58,371] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\kraft\Documents\GitHub\data-science\IBM_machine_learning\.conda\Lib\site-packages\pandas\core\indexes\base.py", line 3791, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 152, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 181, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 7080, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: False

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\kraft\Documents\GitHub\data-science\IBM_machine_learning\.conda\Lib\site-packages\flask\app.py", line 1455, in wsgi_app
    respons